In [44]:
!pip install kaggle

In [45]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"trnmqkhnh","key":"cb388ce728bea6bf29cd7f6ea392ba42"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json
#!kaggle competitions download -c dseb-64-data-preparation-final-project

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [46]:
#!unzip /content/dseb-64-data-preparation-final-project.zip

In [47]:
import pandas as pd
info = pd.read_csv('/content/dseb63_columns_description.csv', encoding='cp1252')
info

,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN
...,...,...,...,...,...
214,217,installments_payments.csv,NUM_INSTALMENT_NUMBER,On which installment we observe payment,NaN
215,218,installments_payments.csv,DAYS_INSTALMENT,When the installment of previous credit was su...,time only relative to the application
216,219,installments_payments.csv,DAYS_ENTRY_PAYMENT,When was the installments of previous credit p...,time only relative to the application
217,220,installments_payments.csv,AMT_INSTALMENT,What was the prescribed installment amount of ...,NaN


In [48]:
row_numbers_to_print = ['AMT_PAYMENT']

for row_number in row_numbers_to_print:
  # Find the description corresponding to the row number
  description = info.loc[info['Row'] == row_number, 'Description'].iloc[0]
  print(f"Description for Row {row_number}: {description}")

Description for Row AMT_PAYMENT: What the client actually paid on previous credit on this installment


Description for Row SK_ID_BUREAU: Description for SK_ID_BUREAU not found\
Description for Row MONTHS_BALANCE: Month of balance relative to application date (-1 means the freshest balance date)\
Description for Row STATUS: Status of Credit Bureau loan during the month (active, closed, DPD0-30,… \[C means closed, X means status unknown, 0 means no DPD, 1 means maximal did during month between 1-30, 2 means DPD 31-60,… 5 means DPD 120+ or sold or written off ] )


%%markdown
\[
\begin{array}{|l|p{10cm}|}
\hline
\textbf{Column Name} & \textbf{Description} \\
\hline
\text{SK\_ID\_BUREAU} & \text{Description for SK\_ID\_BUREAU not found} \\
\hline
\text{CREDIT\_ACTIVE} & \text{Status of the Credit Bureau (CB) reported credits} \\
\hline
\text{CREDIT\_CURRENCY} & \text{Recoded currency of the Credit Bureau credit} \\
\hline
\text{DAYS\_CREDIT} & \text{How many days before current application did client apply for Credit Bureau credit} \\
\hline
\text{CREDIT\_DAY\_OVERDUE} & \text{Number of days past due on CB credit at the time of application for related loan in our sample} \\
\hline
\text{DAYS\_CREDIT\_ENDDATE} & \text{Remaining duration of CB credit (in days) at the time of application in Home Credit} \\
\hline
\text{DAYS\_ENDDATE\_FACT} & \text{Days since CB credit ended at the time of application in Home Credit (only for closed credit)} \\
\hline
\text{AMT\_CREDIT\_MAX\_OVERDUE} & \text{Maximal amount overdue on the Credit Bureau credit so far (at application date of loan in our sample)} \\
\hline
\text{CNT\_CREDIT\_PROLONG} & \text{How many times was the Credit Bureau credit prolonged} \\
\hline
\text{AMT\_CREDIT\_SUM} & \text{Current credit amount for the Credit Bureau credit} \\
\hline
\text{AMT\_CREDIT\_SUM\_DEBT} & \text{Current debt on Credit Bureau credit} \\
\hline
\text{AMT\_CREDIT\_SUM\_LIMIT} & \text{Current credit limit of credit card reported in Credit Bureau} \\
\hline
\text{AMT\_CREDIT\_SUM\_OVERDUE} & \text{Current amount overdue on Credit Bureau credit} \\
\hline
\text{CREDIT\_TYPE} & \text{Type of Credit Bureau credit (Car, cash, ...)} \\
\hline
\text{DAYS\_CREDIT\_UPDATE} & \text{How many days before loan application did last information about the Credit Bureau credit come} \\
\hline
\text{AMT\_ANNUITY} & \text{Loan annuity} \\
\hline
\text{SK\_ID\_CURR} & \text{ID of loan in our sample} \\
\hline
\end{array}
\]


In [49]:
bbal = pd.read_csv('/content/dseb63_final_project_DP_dataset/dseb63_final_project_DP_dataset/dseb63_bureau_balance.csv')
buro = pd.read_csv('/content/dseb63_final_project_DP_dataset/dseb63_final_project_DP_dataset/dseb63_bureau.csv')

In [50]:
print(bbal.shape)
print(buro.shape)

(27299925, 3)
(1465325, 17)


BBAL DATA

In [51]:
# check bbal data
bbal.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [52]:
### FEATURE ENGINEERING

# loan default score
bbal["NUM_STATUS"] = 0
bbal["NUM_STATUS"][bbal["STATUS"] == "X"] = None
bbal["NUM_STATUS"][bbal["STATUS"] == "1"] = 1
bbal["NUM_STATUS"][bbal["STATUS"] == "2"] = 2
bbal["NUM_STATUS"][bbal["STATUS"] == "3"] = 3
bbal["NUM_STATUS"][bbal["STATUS"] == "4"] = 4
bbal["NUM_STATUS"][bbal["STATUS"] == "5"] = 5
bbal["LOAN_SCORE"] = bbal["NUM_STATUS"] / (abs(bbal["MONTHS_BALANCE"]) + 1)
loan_score = bbal.groupby("SK_ID_BUREAU", as_index = False).LOAN_SCORE.sum()
del bbal["NUM_STATUS"]
del bbal["LOAN_SCORE"]

# dummy encoding for STATUS
bbal = pd.get_dummies(bbal, columns = ["STATUS"], prefix = "STATUS")

<ipython-input-52-f7d1a36e58b8>:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  bbal["NUM_STATUS"][bbal["STATUS"] == "X"] = None
<ipython-input-52-f7d1a36e58b8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [53]:
##### FUNCTION FOR COUNTING MISSINGS
def count_missings(data):
    """
    Count missing values in a DataFrame.

    Parameters:
    - data: Input DataFrame

    Returns:
    - DataFrame with columns for total missing values and percentage, sorted in descending order of missing values.
    """
    # Calculate total and percentage of missing values
    total = data.isnull().sum()
    percent = (total / len(data)) * 100

    # Combine into a DataFrame
    table = pd.DataFrame({"Total": total, "Percent": percent})

    # Filter rows with missing values and sort
    table = table.sort_values(by="Total", ascending=False)

    return table

# count missings
nas = count_missings(bbal)
nas.head()

,Total,Percent
SK_ID_BUREAU,0,0.0
MONTHS_BALANCE,0,0.0
STATUS_0,0,0.0
STATUS_1,0,0.0
STATUS_2,0,0.0


In [54]:
### AGGREGATIONS

# total month count
cnt_mon = bbal[["SK_ID_BUREAU", "MONTHS_BALANCE"]].groupby("SK_ID_BUREAU").count()
del bbal["MONTHS_BALANCE"]

# aggregate data
agg_bbal = bbal.groupby("SK_ID_BUREAU").mean()

# add total month count
agg_bbal["MONTH_COUNT"] = cnt_mon

# add loan score
agg_bbal = agg_bbal.merge(loan_score, how = "left", on = "SK_ID_BUREAU")

In [55]:
# count missings
nas = count_missings(agg_bbal)
nas.head()

,Total,Percent
SK_ID_BUREAU,0,0.0
STATUS_0,0,0.0
STATUS_1,0,0.0
STATUS_2,0,0.0
STATUS_3,0,0.0


In [56]:
# clear memory
del bbal
# check data
agg_bbal.head()

,SK_ID_BUREAU,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X,MONTH_COUNT,LOAN_SCORE
0,5001709,0.000000,0.0,0.0,0.0,0.0,0.0,0.886598,0.113402,97,0.0
1,5001710,0.060241,0.0,0.0,0.0,0.0,0.0,0.578313,0.361446,83,0.0
2,5001711,0.750000,0.0,0.0,0.0,0.0,0.0,0.000000,0.250000,4,0.0
3,5001712,0.526316,0.0,0.0,0.0,0.0,0.0,0.473684,0.000000,19,0.0
4,5001713,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,22,0.0


BURO DATA

In [57]:
# check buro data
buro.head()

,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,SK_ID_CURR
0,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,254629
1,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,254629
2,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,254629
3,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,254629
4,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,254629


In [58]:
### MERGE
buro = buro.merge(right = agg_bbal.reset_index(), how = "left", on = "SK_ID_BUREAU")

Calculating the Number of Bureau Loans for each different customer

In [59]:
##### FEATURE ENGINEERING

# number of buro loans
cnt_buro = buro[["SK_ID_CURR", "SK_ID_BUREAU"]].groupby(["SK_ID_CURR"], as_index = False).count()
cnt_buro.columns = ["SK_ID_CURR", "CNT_BURO_LOANS"]
buro = buro.merge(cnt_buro, how = "left", on = "SK_ID_CURR")

Feature Engineering - Amount Ratios

In [60]:
# amount ratios
buro["AMT_SUM_OVERDUE_RATIO_1"] = buro["AMT_CREDIT_SUM_OVERDUE"] / buro["AMT_ANNUITY"]
buro["AMT_SUM_OVERDUE_RATIO_2"] = buro["AMT_CREDIT_SUM_OVERDUE"] / buro["AMT_CREDIT_SUM"]
buro["AMT_MAX_OVERDUE_RATIO_1"] = buro["AMT_CREDIT_MAX_OVERDUE"] / buro["AMT_ANNUITY"]
buro["AMT_MAX_OVERDUE_RATIO_2"] = buro["AMT_CREDIT_MAX_OVERDUE"] / buro["AMT_CREDIT_SUM"]
buro["AMT_SUM_DEBT_RATIO_1"]    = buro["AMT_CREDIT_SUM_DEBT"] / buro["AMT_CREDIT_SUM"]
buro["AMT_SUM_DEBT_RATIO_2"]    = buro["AMT_CREDIT_SUM_DEBT"] / buro["AMT_CREDIT_SUM_LIMIT"]

create logs\
compute the natural logarithm of specified numeric features

In [61]:
import numpy as np
def create_logs(data, features, replace=False):
    if replace:
        data[features] = np.log(data[features].abs() + 1)
    else:
        log_features = {f"LOG_{var}": np.log(data[var].abs() + 1) for var in features}
        data = data.assign(**log_features)
    return data

# Apply log transformation
log_vars = ["AMT_CREDIT_SUM", "AMT_CREDIT_SUM_DEBT", "AMT_CREDIT_SUM_LIMIT", "AMT_CREDIT_SUM_OVERDUE", "AMT_ANNUITY"]
buro = create_logs(buro, log_vars, replace=True)

convert days\
converts day-based features (e.g., DAYS_CREDIT, DAYS_ENDDATE_FACT) into months or other time units by dividing the values by a factor t.\
t: 1 for days, 30 for months, 365 for years

In [62]:
import numpy as np

# Function to convert days to a more interpretable format
def convert_days(data, features, t=12, rounding=True, replace=False):
    for var in features:
        if replace:
            # Avoid negating None values
            data[var] = data[var].apply(lambda x: -x / t if x is not None else None)
            if rounding:
                data[var] = data[var].round()
            # Handle negative values by setting them to None
            data.loc[data[var] < 0, var] = None
        else:
            data["CONVERTED_" + str(var)] = data[var].apply(lambda x: -x / t if x is not None else None)
            if rounding:
                data["CONVERTED_" + str(var)] = data["CONVERTED_" + str(var)].round()
            # Handle negative values by setting them to None
            data.loc[data["CONVERTED_" + str(var)] < 0, "CONVERTED_" + str(var)] = None
    return data

# Convert days for specific variables
day_vars = ["DAYS_CREDIT", "CREDIT_DAY_OVERDUE", "DAYS_CREDIT_ENDDATE", "DAYS_ENDDATE_FACT", "DAYS_CREDIT_UPDATE"]
buro = convert_days(buro, day_vars, t=1, rounding=False, replace=True)

# Recency-weighted loan score
buro["WEIGHTED_LOAN_SCORE"] = np.where(buro["DAYS_CREDIT"] == 0, 0, buro["LOAN_SCORE"] / (buro["DAYS_CREDIT"] / 12))

# Day differences
buro["DAYS_END_DIFF_1"] = buro["DAYS_ENDDATE_FACT"] - buro["DAYS_CREDIT_ENDDATE"]
buro["DAYS_END_DIFF_2"] = buro["DAYS_CREDIT_UPDATE"] - buro["DAYS_CREDIT_ENDDATE"]

# Duration of loans (planned vs actual)
buro["DAYS_DURATION_1"] = buro["DAYS_CREDIT_ENDDATE"] - buro["DAYS_CREDIT"]
buro["DAYS_DURATION_2"] = buro["DAYS_ENDDATE_FACT"] - buro["DAYS_CREDIT"]


number of active buro loans per cus

In [63]:
# number of active buro loans
cnt_buro = buro[["SK_ID_CURR", "CREDIT_ACTIVE"]]
cnt_buro.columns = ["SK_ID_CURR", "CNT_BURO_ACTIVE"]
cnt_buro = cnt_buro[cnt_buro["CNT_BURO_ACTIVE"] == "Active"]
cnt_buro = cnt_buro[["SK_ID_CURR", "CNT_BURO_ACTIVE"]].groupby(["SK_ID_CURR"], as_index = False).count()
buro = buro.merge(cnt_buro, how = "left", on = "SK_ID_CURR")
buro["CNT_BURO_ACTIVE"].fillna(0, inplace = True)

<ipython-input-63-51decad6eb19>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  buro["CNT_BURO_ACTIVE"].fillna(0, inplace = True)


number of closed buro loans per cus

In [64]:
cnt_buro = buro[["SK_ID_CURR", "CREDIT_ACTIVE"]]
cnt_buro.columns = ["SK_ID_CURR", "CNT_BURO_CLOSED"]
cnt_buro = cnt_buro[cnt_buro["CNT_BURO_CLOSED"] == "Closed"]
cnt_buro = cnt_buro[["SK_ID_CURR", "CNT_BURO_CLOSED"]].groupby(["SK_ID_CURR"], as_index = False).count()
buro = buro.merge(cnt_buro, how = "left", on = "SK_ID_CURR")
buro["CNT_BURO_CLOSED"].fillna(0, inplace = True)

<ipython-input-64-a944892ef0c6>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  buro["CNT_BURO_CLOSED"].fillna(0, inplace = True)


number of defaulted buro loans per cus

In [65]:
cnt_buro = buro[["SK_ID_CURR", "CREDIT_ACTIVE"]]
cnt_buro.columns = ["SK_ID_CURR", "CNT_BURO_BAD"]
cnt_buro = cnt_buro[cnt_buro["CNT_BURO_BAD"] == "Bad debt"]
cnt_buro = cnt_buro[["SK_ID_CURR", "CNT_BURO_BAD"]].groupby(["SK_ID_CURR"], as_index = False).count()
buro = buro.merge(cnt_buro, how = "left", on = "SK_ID_CURR")
buro["CNT_BURO_BAD"].fillna(0, inplace = True)

<ipython-input-65-99c90fdbf6a6>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  buro["CNT_BURO_BAD"].fillna(0, inplace = True)


dummy thay vì onehot

In [66]:
# dummy encodnig for factors
buro = pd.get_dummies(buro, drop_first = True)

In [67]:
# count missings
nas = count_missings(buro)
nas.head()

,Total,Percent
AMT_MAX_OVERDUE_RATIO_1,1403790,95.800590
AMT_SUM_OVERDUE_RATIO_1,1296975,88.511081
AMT_SUM_DEBT_RATIO_2,1135898,77.518503
AMT_ANNUITY,1130013,77.116885
AMT_MAX_OVERDUE_RATIO_2,971919,66.327879


aggregations

Numeric Features: Compute mean, std, min, and max for all numeric columns grouped by SK_ID_CURR.

Categorical Features: Compute mode and number of unique values (nunique) for all categorical columns grouped by SK_ID_CURR.

In [68]:
import pandas as pd
import scipy.stats

##### FUNCTION FOR AGGREGATING DATA
def aggregate_data(data, id_var, label = None):


    ### SEPARATE FEATURES

    # display info
    print("- Preparing the dataset...")

    # find factors
    data_factors = [f for f in data.columns if data[f].dtype == "object"]

    # partition subsets
    num_data = data[list(set(data.columns) - set(data_factors))]
    fac_data = data[[id_var] + data_factors]

    # display info
    num_facs = fac_data.shape[1] - 1
    num_nums = num_data.shape[1] - 1
    print("- Extracted %.0f factors and %.0f numerics..." % (num_facs, num_nums))


    ##### AGGREGATION

    # aggregate numerics
    if (num_nums > 0):
        print("- Aggregating numeric features...")
        num_data = num_data.groupby(id_var).agg(["mean", "std", "min", "max"])
        num_data.columns = ["_".join(col).strip() for col in num_data.columns.values]
        num_data = num_data.sort_index()

    # aggregate factors
    if (num_facs > 0):
        print("- Aggregating factor features...")
        fac_data = fac_data.groupby(id_var).agg([("mode",   lambda x: scipy.stats.mode(x)[0][0]),
                                                 ("unique", lambda x: x.nunique())])
        fac_data.columns = ["_".join(col).strip() for col in fac_data.columns.values]
        fac_data = fac_data.sort_index()


    ##### MERGER

    # merge numerics and factors
    if ((num_facs > 0) & (num_nums > 0)):
        agg_data = pd.concat([num_data, fac_data], axis = 1)

    # use factors only
    if ((num_facs > 0) & (num_nums == 0)):
        agg_data = fac_data

    # use numerics only
    if ((num_facs == 0) & (num_nums > 0)):
        agg_data = num_data


    ##### LAST STEPS

    # update labels
    if label != None:
        agg_data.columns = [label + "_" + str(col) for col in agg_data.columns]

    # display info
    print("- Final dimensions:", agg_data.shape)

    # return dataset
    return agg_data

### AGGREGATIONS

# count previous buro loans
cnt_buro = buro[["SK_ID_CURR", "SK_ID_BUREAU"]].groupby("SK_ID_CURR").count()
del buro["SK_ID_BUREAU"]

# aggregate data
agg_buro = aggregate_data(buro, id_var = "SK_ID_CURR", label = "buro")

# add buro loan count
agg_buro["buro_BURO_COUNT"] = cnt_buro

# clean up
omits = ["WEIGHTED_LOAN_SCORE"]
for var in omits:
    del agg_buro["buro_" + str(var) + "_std"]
    del agg_buro["buro_" + str(var) + "_min"]
    del agg_buro["buro_" + str(var) + "_max"]

- Preparing the dataset...
- Extracted 0 factors and 58 numerics...
- Aggregating numeric features...
- Final dimensions: (263491, 232)


In [69]:
# count missings
nas = count_missings(agg_buro)
nas.head()

,Total,Percent
buro_AMT_SUM_DEBT_RATIO_2_std,260130,98.724435
buro_AMT_MAX_OVERDUE_RATIO_1_std,251859,95.585428
buro_AMT_MAX_OVERDUE_RATIO_1_min,228193,86.603717
buro_AMT_MAX_OVERDUE_RATIO_1_max,228193,86.603717
buro_AMT_MAX_OVERDUE_RATIO_1_mean,228193,86.603717


In [70]:
# check data
agg_buro.head()

,buro_DAYS_END_DIFF_1_mean,buro_DAYS_END_DIFF_1_std,buro_DAYS_END_DIFF_1_min,buro_DAYS_END_DIFF_1_max,buro_CREDIT_TYPE_Mortgage_mean,buro_CREDIT_TYPE_Mortgage_std,buro_CREDIT_TYPE_Mortgage_min,buro_CREDIT_TYPE_Mortgage_max,buro_AMT_CREDIT_SUM_DEBT_mean,buro_AMT_CREDIT_SUM_DEBT_std,...,buro_CREDIT_TYPE_Credit card_max,buro_STATUS_1_mean,buro_STATUS_1_std,buro_STATUS_1_min,buro_STATUS_1_max,buro_index_mean,buro_index_std,buro_index_min,buro_index_max,buro_BURO_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,0.0,NaN,False,False,11.258904,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,NaN,NaN,NaN,NaN,0.0,NaN,False,False,0.000000,NaN,...,False,0.0,NaN,0.0,0.0,408662.0,NaN,408662.0,408662.0,1
2,21.0,50.186320,-30.0,88.0,0.0,0.0,False,False,3.586919,5.558466,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
3,32.5,37.669616,0.0,69.0,0.0,0.0,False,False,4.438169,5.619434,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
4,456.0,NaN,456.0,456.0,0.0,0.0,False,False,6.432776,7.053138,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6


In [71]:
# prompt: check null agg_buro

# Check for null values in agg_buro
null_counts = agg_buro.isnull().sum()
null_counts

,0
buro_DAYS_END_DIFF_1_mean,43104
buro_DAYS_END_DIFF_1_std,100594
buro_DAYS_END_DIFF_1_min,43104
buro_DAYS_END_DIFF_1_max,43104
buro_CREDIT_TYPE_Mortgage_mean,0
...,...
buro_index_mean,171260
buro_index_std,183446
buro_index_min,171260
buro_index_max,171260


In [72]:
# prompt: fill 0

# Fill NaN values with 0 in agg_buro DataFrame
agg_buro.fillna(0, inplace=True)

# Verify that there are no more NaN values
null_counts = agg_buro.isnull().sum()
null_counts

,0
buro_DAYS_END_DIFF_1_mean,0
buro_DAYS_END_DIFF_1_std,0
buro_DAYS_END_DIFF_1_min,0
buro_DAYS_END_DIFF_1_max,0
buro_CREDIT_TYPE_Mortgage_mean,0
...,...
buro_index_mean,0
buro_index_std,0
buro_index_min,0
buro_index_max,0


In [ ]:
# prompt: to csv

# Assuming 'agg_buro' is the DataFrame you want to save
agg_buro.to_csv('fin_buro.csv', index=False)